In [1]:
import time
iss = 0

def make_submit(pred):
    global iss
    submit = pd.DataFrame()
    submit['id'] = data_test.index
    submit['skilled'] = pred
    iss += 1
    submit.to_csv('submissions/submission{}-{}.csv'.format(''.join(time.ctime().split()[2]), iss), index=False)
    print('last: ', 'submissions/submission{}-{}.csv'.format(''.join(time.ctime().split()[2]), iss))

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
import sklearn 
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import catboost
import lightgbm
import xgboost
import eli5
from eli5.sklearn import PermutationImportance

In [3]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, Ridge
from sklearn.model_selection import ShuffleSplit, KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [4]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [5]:
from annoy import AnnoyIndex

Plan
-------

+ Tune models xgb/lgbm with Hyperopt +++ wirite modul for night work 

+ Feature interations (Poly) +++ wirite modul for random -+*/ between important columns

+ KNN features (top 1 must be)

+ Staking, boosting, blending

In [6]:
data_train = pd.read_csv('data/data_train_v6_1.csv')
data_test = pd.read_csv('data/data_test_v6_1.csv')
data_train.set_index('id', inplace=True)
data_test.set_index('id', inplace=True)

In [7]:
target = data_train.skilled
data_train = data_train.drop(['skilled', 'player_team', 'winner_team','Unnamed: 0' ,'Unnamed: 0.1','Unnamed: 0.1.1', 'Unnamed: 0.1.1.1','Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1','Unnamed: 0.1.1.1.1.1.1.1'], axis=1)
data_test = data_test.drop(['Unnamed: 0','player_team', 'winner_team', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1','Unnamed: 0.1.1.1.1.1.1.1'], axis=1)

In [8]:
X = data_train
x_test = data_test
y = target

<hr>

# KNN features

In [11]:
annoy_data_train = X
annoy_index_train = X.index.get_values()

In [16]:
annoy_features = X.columns

In [17]:
annoy_vector_size = len(annoy_features)
annoy = AnnoyIndex(annoy_vector_size)

In [18]:
for vec, i in tqdm_notebook(zip(annoy_data_train.get_values(), annoy_index_train)):
    annoy.add_item(i, vec)

In [19]:
annoy.build(100)

True

In [101]:
row = X.iloc[:1,:]
vec = row[annoy_features].get_values()[0]
indexes = annoy.get_nns_by_vector(vec, 20)
    
if row.index[0] in indexes:
    indexes.remove(row.index[0])
else:
    indexes = indexes[:-1] 
    
ys = [target[ind] for ind in indexes if ind in X.index]

In [92]:
k = 0
def get_nn_info(row):
    global k
    if k%1000==0: print(k)
    k+=1
    
    vec = row[annoy_features].get_values()[0]
    indexes = annoy.get_nns_by_vector(vec, 501)
    
    if row.index[0] in indexes:
        indexes.remove(row.index[0])
    else:
        indexes = indexes[:-1] 
    
    ys = [target[ind] for ind in indexes if ind in X.index]

    info = {
        "nn_mean_5": np.mean(ys[:5]),
        "nn_median_5": np.median(ys[:5]),
        "nn_mean_10": np.mean(ys[:10]),
        "nn_median_10": np.median(ys[:10]),
        "nn_mean_20": np.mean(ys),
        "nn_median_20": np.median(ys)
    }
    return pd.Series(info)

In [38]:
%%time
X = pd.concat([X, X.apply(get_nn_info, axis="columns")], axis=1)
x_test = pd.concat([x_test, x_test.apply(get_nn_info, axis="columns")], axis=1)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
CPU times: user 56min 25s, sys: 15.4 s, total: 56min 41s
Wall time: 56min 41s


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
k_fold = StratifiedKFold(n_splits=5, shuffle=True)

<hr>

### Moldeling

In [40]:
clf_1 = xgboost.XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=4, min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.005, objective= 'binary:logistic', nthread=-1, scale_pos_weight=1, seed=27)
clf_2 =  lightgbm.LGBMClassifier(n_estimators=1000, learning_rate=0.1, reg_alpha=10, max_depth=6, min_child_weight=4, n_jobs=5)  
clf_3  =  GradientBoostingClassifier()
clf_4 = catboost.CatBoostClassifier(n_estimators=1200, max_depth=6)

In [41]:
# cv = cross_val_score(clf_2, X, y, cv = k_fold, scoring='accuracy', n_jobs=-1)
# cv.mean()

In [42]:
%%time
clf_2.fit(X_train, y_train)

CPU times: user 6min 52s, sys: 7.23 s, total: 6min 59s
Wall time: 1min 32s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=6,
        min_child_samples=20, min_child_weight=4, min_split_gain=0.0,
        n_estimators=1000, n_jobs=5, num_leaves=31, objective=None,
        random_state=None, reg_alpha=10, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

<hr>

Accuracy
------
+ 0.8187996844468718 - gpm with 5/10/20 train+test
+ 0.817282602099642 - gpm with 50/100/200/500 train+test

In [48]:
accuracy_score(y_test, clf_2.predict(X_test))

0.817282602099642

<hr>

In [86]:
a = X[target[X.index] == 0][annoy_features].values[1].astype('int')
b = X[target[X.index] == 0][annoy_features].values[1]

### Submit

In [23]:
clf_2.fit(X,y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=6,
        min_child_samples=20, min_child_weight=4, min_split_gain=0.0,
        n_estimators=1000, n_jobs=5, num_leaves=31, objective=None,
        random_state=None, reg_alpha=10, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [19]:
pred = eclf1.predict(x_test)

In [21]:
make_submit(pred)

last:  submissions/submission30-3.csv


<hr> 

In [22]:
df1 = pd.read_csv('submissions/submission30-1.csv')
df2 = pd.read_csv('submissions/submission26-1.csv')

In [27]:
df2.shape

(43265, 2)

In [23]:
accuracy_score(df1['skilled'], df2['skilled'])

0.9344967063446203

<hr>

## Poly

# Models

## Tune

  + __ -<br>
  + __booster__ -  gbtree, gblinear or dart <br>
  + __learning_rate__ - [0,1] <br>
  + __gamma__ - [0,inf] he larger gamma is, the more conservative the algorithm will be <br>
  + __max_depth__ [0,inf]  <br>
  + __min_child_weight__ - [0, inf] The larger min_child_weight is, the more conservative the algorithm will be. <br>
  + __max_delta_step__  [0,inf] [1-10]  <br>
  + __subsample__ - (0,1] 0,5 -+ <br>
  + __lambda__ - L2 reg  Increasing this value will make model more conservative.<br>
  + __alpha__ - L1 reg  Increasing this value will make model more conservative.
  + __tree_method__  - auto, exact, approx, hist<br>
  + ___objective__ - binary:logistic, binary:logitraw, binary:hinge
  +__colsample_bytree__ - (0,1]

In [14]:
#12 8 9 - 0.8197402755021542 -  0.819253
#'max_depth': 10, 'min_child_weight': 4, 'reg_alpha': '10.000' - 0.819527883973542
#'max_depth': 7, 'min_child_weight': 7, 'reg_alpha': '11.000' -  0.8196492505613204
#'max_depth': 6, 'min_child_weight': 6, 'reg_alpha': '8.000 - 0.8204077917349354   - 	0.818676

In [ ]:
clf_1 =  lightgbm.LGBMClassifier(n_estimators=1500, learning_rate=0.1,
                                 max_depth=12, reg_alpha=9, min_child_weight=8, n_jobs=-1)
clf_2  = xgboost.XGBClassifier(max_depth=12, learning_rate=0.07,n_estimators=1500,
                               min_child_weight=11, gamma=1, subsample=0.6, colsample_bytree=0.3, reg_alpha=3.5,objective='binary:logitraw',nthread=-1 ,scale_pos_weight=2.2)